In [ ]:
from init_notebook import *

In [ ]:
class Fractal:
    def __init__(
            self,
            points: Union[int, Tuple[Tuple[float, float]]],
            ratio: float = .5,
            start_point: Tuple[float, float] = (0, 0), 
            steps: int = 1,
    ):
        if isinstance(points, int):
            arr = []
            for i in range(points):
                t = i / points * math.pi * 2
                arr.append((math.sin(t), math.cos(t)))
            points = tuple(arr)
        self.points = points
        self.ratio = ratio
        self.start_point = start_point
        self.steps = steps

    def iter_points(self, start_point: Optional[Tuple[int, int]] = None, steps: Optional[int] = None):
        if start_point is None:
            start_point = self.start_point
        if steps is None:
            steps = self.steps
        for p in self.points:
            px = p[0] - start_point[0]
            py = p[1] - start_point[1]
            #d = max(10e-7, math.sqrt(px*px + py*py))
            #nx = px / d
            #ny = py / d
            new_p = (
                start_point[0] + px * self.ratio,
                start_point[1] + py * self.ratio,
            )
            yield new_p
            if steps > 1:
                yield from self.iter_points(new_p, steps=steps - 1)

    def to_pil(self, size: Tuple[int, int], padding: float = .1):
        image = self.to_numpy(size=size)
        return VF.to_pil_image(torch.from_numpy(image[::-1].copy()).unsqueeze(0))
        
    def to_numpy(self, size: Tuple[int, int], padding: float = .1):
        points = list(self.iter_points())
        min_x = min(p[0] for p in points) - padding
        min_y = min(p[1] for p in points) - padding
        max_x = max(p[0] for p in points) + padding
        max_y = max(p[1] for p in points) + padding
        min_x = min(min_x, min_y)
        max_x = max(max_x, max_y)
        min_y, max_y = min_x, max_x
        image = np.zeros((size[1], size[0]))
        #print(min_x, min_y, max_x, max_y, points)
        for px, py in points:
            x = int(((px - min_x) / (max_x - min_x)) * size[0])
            y = int(((py - min_y) / (max_y - min_y)) * size[1])
            if 0 <= x < size[0] and 0 <= y < size[1]:
                #print((px, py), (x, y))
                image[y, x] += 1
        #image = image / steps
        image = image.clip(0, 1)
        #print(image.max())
        return image

Fractal(
    3,#((-1, -1), (0, 1), (1, -1)),
    ratio=3/2,
    #ratio=6/10,
    steps=3,
).to_pil(size=(300, 300)).resize((600, 600))

In [ ]:
def plot_fractals(
    fractals_iterable,
    size=(128, 128),
    nrow=8,
):
    grid = []
    labels = []
    for f in fractals_iterable:
        grid.append(f.to_pil(size))
        labels.append(f"r={f.ratio}")
        
    display(VF.to_pil_image(
        make_grid_labeled([VF.to_tensor(c) for c in grid], nrow=nrow, labels=labels)
    ))

N = 12
S = (3, 7)
plot_fractals((
    Fractal(
        s,
        ratio=(i+1)/N,
        #ratio=6/10,
        steps=6,
    )
    for s in range(S[0], S[1] + 1)
    for i in range(N)
), nrow=N)
        

In [ ]:
N = 12
S = (3, 9)
plot_fractals((
    Fractal(
        s,
        #ratio=(i+1)/N,
        ratio=(s)/(s+3),
        steps=6 + max(0, 6-s),
    )
    for s in range(S[0], S[1] + 1)
    #for i in range(N)
), nrow=N, size=(512, 512))

In [ ]:
N = 12
S = (3, 9)
plot_fractals(
    (
        Fractal(
            s,
            #ratio=(i+1)/N,
            (s-2)/(s+1),
            steps=6 + max(0, 6-s),
        )
        for s in range(S[0], S[1] + 1)
        #for i in range(N)
    ), 
    nrow=N, 
    size=(512, 512),
)

In [ ]:
N=8*8
R=(.0, 2)
plot_fractals(
    (
        Fractal(
            5,
            ratio=i/N*(R[1]-R[0])+R[0],
            steps=6+2,# + max(0, 6-3),
        )
        for i in range(N)
    ), 
    nrow=8, 
    size=(512, 512),
)

In [ ]:
import scipy

x = np.linspace(0, 10, 100)
df = pd.DataFrame({
    #"x": x,
    **{
        f"s{i}": np.pow(x, i) / scipy.special.factorial(i) * math.pow(-1, i+1)
        for i in range(1, 10)
    }
})
df["sum"] = df.sum(axis=1)
df.plot.line()